In [1]:
import requests
import hashlib
from datetime import datetime
import time
import random
import os
import csv
from bs4 import BeautifulSoup
import re 
import pandas as pd

# Parsing functions

In [2]:
def parse_listing_html(text,file_name):
    #define the html text
    page_html = BeautifulSoup(text, 'html.parser')
    
    if deleted_listing(page_html):
        print('listing {} has been deleted'.format(file_name))
        return
    
    if page_html.find('meta',attrs={'property':'og:url'}) is None: 
        return
    else:
        listing_url = page_html.find('meta',attrs={'property':'og:url'}).attrs['content']
    try:
        listing_id = page_html.findAll('p', class_='postinginfo')[1].text[9:]

        listing_title = page_html.find('span',attrs={'id':'titletextonly'}).text
        
        listing_nh = page_html.find('small').text[2:-1]
        
        listing_city = page_html.find('meta',attrs={'name':'geo.placename'}).attrs['content']
        
        listing_date = page_html.find('time', class_='date timeago')['datetime']
        
        if page_html.find('span', class_='price') is None: 
            return
        else:
            listing_price = int(page_html.find('span', class_='price').text.strip('$,').replace(",",""))

        
        listing_bedrooms = page_html.find('span', class_='shared-line-bubble').text.split('BR')[0]
        
        listing_bathrooms = page_html.find('span', class_='shared-line-bubble').text.split('/')[1][:-2]
        
        # Extract sqft info from bubbles class and use regex to find the digits ending in ft2
        bubbles_sqft = page_html.findAll('span', class_='shared-line-bubble')
        listing_sqft = None
        if len(bubbles_sqft) >= 2:
            x = re.search("[0-9]+ft2$", bubbles_sqft[1].text)
            if x:
                 listing_sqft = x.string[:-3]

        listing_address_element = page_html.find('div', class_='mapaddress')
        listing_address = listing_address_element and listing_address_element.text

        listing_info = page_html.findAll('p', class_='attrgroup')[1].text

        listing_body = page_html.find('section',attrs={'id':'postingbody'}).text[30:]

        listing_first_image = page_html.find('div',class_='slide first visible').img['src']
    except Exception as e:
        print('Error parsing url: {} file: {}'.format(listing_url, file_name))
        raise e

    return {
        'listing_title': listing_title,
        'listing_nh': listing_nh,
        'listing_city':listing_city,
        'listing_date':listing_date,
        'listing_price': listing_price,
        'listing_bedrooms':listing_bedrooms,
        'listing_bathrooms':listing_bathrooms,
        'listing_sqft':listing_sqft,
        'listing_address':listing_address,
        'listing_info':listing_info,
        'listing_body':listing_body,
        'listing_id':listing_id,
        'listing_url':listing_url,
        'listing_first_image':listing_first_image
       }


def deleted_listing(page_html):
    h2 = page_html.find('h2')
    return h2 and h2.text == ' This posting has been deleted by its author. '

# Create df from dictionary

In [3]:
list_of_dicts = []
path = "/Users/pandabear/springboard/CapstoneTwoProject/data/raw"
dir_list = os.listdir(path)


### Testing parsing for one folder scraped by playwright
# path_testing = '/Users/pandabear/springboard/CapstoneTwoProject/data/raw/2022-08-17T22:01:42'
# file_list = os.listdir(path_testing)
# for file in file_list:
#     if len(file) > 15:
#         with open(path_testing + '/' + file, 'r') as f:
#             text = f.read()
#             parsed_data = parse_listing_html(text, file)
#             if parsed_data:
#                 list_of_dicts.append(parsed_data)

# listing_df = pd.DataFrame(list_of_dicts)
### Testing code


for folder in dir_list:
    if not folder.startswith('.'):
        path_html = path + '/' + folder
        file_list = os.listdir(path_html)
#         print(file_list)
        
        for file in file_list:
            if len(file) > 15:
                with open(path_html + '/' + file, 'r') as f:
                    text = f.read()
                    parsed_data = parse_listing_html(text, file)
                    if parsed_data:
                        list_of_dicts.append(parsed_data)
#                 print('Number of parsing errors: {}'.format(count_parsing_error))
        new_listing_df = pd.DataFrame(list_of_dicts)


listing bd2f42238071ca5af0f51ff5d03e5d735d241e51dc0fd7b48bfdb285ab34ae70 has been deleted
listing 16684e5f69795abf39ccb1636e4643f54c3ed25e9f630564874e35ab00c50643 has been deleted
listing bbdf06f159200b28adca3d0e1a10fd4804734f275fe81b22210294854705bb7b has been deleted
listing 090c7d6ec91186068b21b2cb64e1cf68529cbbe00788764da3f290c2ac5a4a89 has been deleted
listing edd31a8f6a013d2b4681cad06321a0954c2bc16af0768af73c677bc471381e1e has been deleted
listing 2a17a6e8e33a2deff19eccb9c5ab48f9efdc1c81c4665f47b864ac4e884c83e1 has been deleted
listing 4dff65c137f062e598d422ea8ab2ace593ee615c4ff5f1aea8c5a4bcbdd84765 has been deleted
listing efac3de9f6c47188415f2cc3a45e8f556a2f0c8775d9e247889028c67e16bdbd has been deleted
listing 4c126db3423111b873047b3ecb3428f86fdf20ba03039f492624a36b0f145f5c has been deleted
listing 0841adc649ce290e973586ac8c7c4c6b445545815a1dffca48f3c0e14e2bfe37 has been deleted
listing aa17c81011918fb206b5c9b67a36370e2e43e3c4471e6e5e5693596b8f247265 has been deleted
listing a5

listing a68c1039cb2dc802ba720d77b34fc526bcd4962e6df0e12565564ad18a025289 has been deleted
listing b3753398048d5d8726cbc47e53093b34b3ef982edd05429a91e90a34624da8d8 has been deleted
listing 369c4eb7a669053671ef9ef209707c5f516f6d60d32e6ca070426157893dbaa5 has been deleted
listing 1467836236ffa78ca1002ed823e078a651ebe6c80b19976ee76143a4c6f965f5 has been deleted
listing dd58812ccf3a9011d0e8788327a1116b5fc369b3cc913aa8e80e26cae53318fc has been deleted
listing a05111172bd8389d1662871b37152457d9af5c6fc053798f5cb6a5fd2cd193dc has been deleted
listing b28d5d548c7d92d87f6db49089824954285e559804c82f6d2cc65d78a224be0f has been deleted
listing 592ebbcf420b0ecabed960bf03c0715500749bf2c8b96caf105cfc4028abf7c3 has been deleted
listing da39ff7a32bcaad356bba1272d6e0abbf291d277ffc57f6e5403d462a50e78d7 has been deleted
listing 1ba5a25f1b37f8a9b2bf2d163205b86cec58ce15c5689405a4411f0c5520c50c has been deleted
listing 576be37c10847e4fba2a244c0c92c8d005135d82d8bf8c365b835664deb53037 has been deleted
listing 5e

listing e123cff1c76ddb5f06b402d9725455eb87d4cd9b146275bdbd322dcbbf8b1b99 has been deleted
listing 8984b518bd86f38ec4f06d5daf3437ee88f74953992a2e3cb6615f34d4b77996 has been deleted
listing 7f188bca4f43b81f02504fa29163f8107596588e402720635fd51b2ed41d4abe has been deleted
listing 21f4982d1d59b17f215a7c0fa3027fdb22bf6101afcb1841af22decd87a7440e has been deleted
listing 11927f6af6399c367fc23138bee71b3e3fd04f791c4a9b96f824ecd7e3b4687d has been deleted
listing aaf150ca7fa9feb3eb5a4750c0ba2514c4dd96fdf5483d2b700044561c2a0e81 has been deleted
listing 96a6a0e0a5be00def85f1704eb075fa7ca9f98b4ec4b9e9e0a403f93f737421b has been deleted
listing bcdc52223e7a6c4d38db1a8fb7b99f4a805c1101fdd056a6ad7453a8d46c3b48 has been deleted
listing 48da75b3444770546c257d9db9315f7abdd7986319b205928c8bf78251b60f50 has been deleted
listing 404fd8d872643e7f9e83e1fcde5350c002442fc4587607655c6a9bde81161571 has been deleted
listing 44790c132730e1849460b88808b00a7cd3c45288faf1d7796e280391f1d27ab7 has been deleted
listing e7

listing 377f625896f7ac1f2ec392d8d4f1946574d1c8c148b1aec628f3efe85cb83162 has been deleted
listing 2178d5fc2724fbe0a78a5d144694edbfdfb34c2e1237e7da430cfb57f091c9a8 has been deleted
listing 7612f1497b8bc232f2151e239dffcb67df54fc1dc1d21d25483963bc9a151122 has been deleted
listing 08607b36a249c453c579bfe37b4b7daed0b5d2279d0cd0e596872dae6eed3551 has been deleted
listing 6ec2a9dc508785d72faf3d6293eb7eac5d09a737d73290edab2e26e99e2c5122 has been deleted
listing 0e5b09b9f8ae88dcdd9680bdfaef428c31a9fc3ff723355a741c71b55206c079 has been deleted
listing 65dbc4185a7512d5c6e93b3a5ab2d3abb3b066bf44024fe8af01bc637dcd8d6d has been deleted
listing bfa5fbd49f3ffc223705f3a4f528511276bac1f20e72e43a91790944c7041839 has been deleted
listing 2e943ad3bb633428f774da0ace9e5925e2facb7681d01ff83ade8d19ea3b042d has been deleted
listing bfa0aa60b452fde438a08225315002da7d12dc190b94ac710fcf9b7b3399afbf has been deleted
listing fe331e6b1c6706af737690ca3ea5a38b54ebaef22a41fc57d9b4821cdc9e49c2 has been deleted
listing b2

listing cecc2a7098a964cc9e905397bc8e9b905ae92e8315f07a969e0fdea610762250 has been deleted
listing 08d3a1cd826dd50c9860bca263a5cfbd375e558af9708ab8edd07a06638de8c2 has been deleted
listing 46d5c7208f9916c09d5fa8eec50afe823a98b0c093c9849af9091d2d903304b3 has been deleted
listing 6f52cd11edfa7c5c2ca65323e08b4d04f568078702d2e0d2f1fca9764bbd8886 has been deleted
listing 3390516f9f948c10dc2d29554a7845ae90afc8b9ff834775244582930dd83548 has been deleted
listing 8851201d5dde84d121303a275294812bcd2ad635142507a8fcd232e256fddfb5 has been deleted
listing 7250089eceaa3ef1b053ca8142bb067ef9ca953905894dbedabfcfa6f2408df8 has been deleted
listing 88c222339b8210a6f004faad5ccb45bfda10dacb8b3061251f90edc8a9db2aae has been deleted
listing 4c1d0a667d44338234ab7e96232a87004edbe4bda742b0d0aea798c5c763a7ef has been deleted
listing 3da27d7fbe39f08bdaaf17e5c9adeaab7a69341eb22a030ead52c183deba3bfc has been deleted
listing 4244483fe447b378ec600927850540cd5cea99862c858ef2e1b788cfc16b4c75 has been deleted
listing b9

...

listing f1791febc94d5dfb190d2e204890fad7d706f41563fb14ce19332b171075e4a5 has been deleted
listing 92f52f3e1e8363a209b791d79cf14a87cceb9437e23897b2628351da28fc44d9 has been deleted
listing 8708ef63385978e05441ef9980e52a6d883349c6c06966d260e48b71361f2eb7 has been deleted
listing 48315025f1891fb878c58bab3231d7eed29b1a2e6c9930635aca70505cfcb0d6 has been deleted
listing 6eaacb36a9b00c59ccf506424ad79f66b7abfa238209ae6019699ec8c81ac01a has been deleted
listing 065dcac2ee9aed33a266edef1eb2d083974d1cb063727a18b6e496f0c1168ac4 has been deleted
listing 9ae83cd4f472e8773b2a6000b85ac8aa84a2695cf4c48be1873f19cf423d2aea has been deleted
listing 0a79c34ef4c1f20e864dd73bc8f198f80369a44ffbeee853b86a9e4a8203d886 has been deleted
listing 0b6b1a5b3b2d60cb6ce13c92b4c7494f529c833084a016f8faca244dd45baf52 has been deleted
listing 8581d54d778b1deb79ee21232a64c5f03265e72fd39a29072251ca286842b21b has been deleted
listing e038a4c497f59f6931f4032572db07b6e11cd0ade03bfbe1ff6a3bc5b7a39c77 has been deleted
listing 1f

listing 5c29b1cbbca5ac465a8f07c9d402a8b8b58420795108b26afaf554d9c31415c5 has been deleted
listing 6959006c8ae6fa6fba94ab879d833656c18200c20a1187c356b0ef48e9bf1f9c has been deleted
listing 7f04ce314cee448407e6e7fd07d6f7f64b44f58adc24b1499c33353c96ca633b has been deleted
listing 95eb97883127e5bcd98ba7059219c5835e69bb54fa506c9bcd99f267ccef909d has been deleted
listing 338ec271b95db1d89824dd27dd295ddd8958fd5018ae54c6a73726218b22e637 has been deleted
listing 49b225a42a25592c3ce3f1857b1d1256427281fb2f8d58924a1034103911247e has been deleted
listing 0d19459886430aeeae7c82d5aed16243ef1f2342cb3c3dc0df149b18c32dacb8 has been deleted
listing 029f8c137d798b9903f85c36904780f19c5d9b2b73f144d1eb4da0f0c2a25ce5 has been deleted
listing f9f277b1338607829a24058b3e31f1ee576fc78f095a7d397851059f6ffdb6f7 has been deleted
listing bb189adba40267cec02a90e026ccc61d7e504307e9065f7d418289f9dd66dddb has been deleted
listing 424d941bcbd007e00b5184bf69b2cb53cdc50e286f0df963ac77343d1f4cb43b has been deleted
listing b0

listing b2c0e07cb0f5183916d4727fc9e0c3db3bee6cc063acd7306e96d7f0fd8d2371 has been deleted
listing c949700d49c89e1778a2667520379f83445e433a548a09a7372f989c9096f8f7 has been deleted
listing 9a27a7207dc42b136575c9eb2ce5cc83f3d7bc149c5e624fd6a0ddaa0cb5e7e8 has been deleted
listing 36be80fb5323f16355dca6e001a908ce0c75b228563ed9250709d19915faf1aa has been deleted
listing 69bd32fd75660e7b99252800f1d324dcb027d33cf47adfcae9474dd44d929552 has been deleted
listing cc8afbb711371d45f0c93ab2829fa2def78806e6f6250fdea28f4b69f8e327ad has been deleted
listing 36d59857ae2506459b202853b5ea3d27e61a288e970c124c17d2f0ed1f8290c2 has been deleted
listing f2b823f0908a8f60b23ad80c2cf4721bf9c2a219e94ef4637173ad5f40ab9a3e has been deleted
listing 67b79b85abaa1451fc967c1f977c66d5baa36409f2880439de98c26dd3ad5c18 has been deleted
listing a04411d8e47b6784909dcdcf7a89dcceed719bfe1c3dfa7bde1addaaa1a4168d has been deleted
listing e3c40fa881bb51267ca388a4e553150dd75ce970b9ad14d35e565fc58f8756ef has been deleted
listing cc

listing f3783af314f1c46bab5c31ef0a82265e21b1ff693439d36da4ef7316eed33fc8 has been deleted
listing a31f717b1d3c5404f1cb8fda9f576137063c0b719f9793261a1a0505fdf7fa06 has been deleted
listing 84b50bca30de9b62b2ff94c48ed113308615edee963ea3119032bc5b75ada52c has been deleted
listing 1f8c44d76b23b95d4ff42c5fed779d6e9c677e1ee0fc350958e18162f45bd412 has been deleted
listing 987e75b32b3d751b070c83cc2855659807433793a77df63ea40a9292646f2ca4 has been deleted
listing cf42ff0aced97a550cc870b9e09f928062978301f7bd92af50999d39200454cc has been deleted
listing 8392e8c2cd3c15623552d03ec9e5beca3be48234199c5861761fcc403c6da879 has been deleted
listing bb9ab8c215e0a6e3e927f289748422e76c6afaf746da91540b94124db7d7ac9c has been deleted
listing cd048132eb53eb5d1270eac64160c69b2097c42d590a69859df161cec52f7c21 has been deleted
listing 4760f335b0615f6d7dc6919950c676b5b4feba92d0a5cec1d8eab038056a9dfb has been deleted
listing 68c26b3fb54b3e73c10b325fe76a6990a024bec3f47de741067714482b743707 has been deleted
listing 26

listing e9941aac8157ad64ad07b49a085b64e10fc03c5b51f0c4b09fdbae0967d9bf5e has been deleted
listing 91335e0df36144098b9ddd809000f38505695b3e45390878322c98b783bdcfe9 has been deleted
listing 6808af810a97f57b6874f06fb35935787059ad98fa6545ec92bc6b51358a9d63 has been deleted
listing 8c3286a728851cedfc6f061a6922156bcfa3856a0b07cc9232495b8c59bc7f5c has been deleted
listing 2d7ec4469466b3447d1a28eb60268fc4424a75eaa18ebebb2b5c9f86dabbf6b1 has been deleted
listing 777974375f18401f7d4cdb1ec3f0b00249e833aeb526ad95668e84346abe0dbb has been deleted
listing e8a65403bcc9b4cc535884664ef5013529fa64662a99beb60f943e486e75dedf has been deleted
listing e5925c074fe7f5f4e879eb7e1477dcfecee5146b7dd70eaca8ce995aeb813d33 has been deleted
listing e0699c7e4b09c7c5302b8438b2c43b9c31577184a9ac8d65e83fa89dd6c27128 has been deleted
listing 64a0bde5db88cffe6619552c858160df3aa37330e349663de094940d1e0cb1ee has been deleted
listing d849f40fa1a9007daa3c24275d25a4c0eb3149ee157e5c9f1afa579909d0b5fb has been deleted
listing 01

In [4]:
new_listing_df.set_index('listing_id', inplace = True)
new_listing_df.head()

,listing_title,listing_nh,listing_city,listing_date,listing_price,listing_bedrooms,listing_bathrooms,listing_sqft,listing_address,listing_info,listing_body,listing_url,listing_first_image
listing_id,,,,,,,,,,,,,
7539510462,1x1 w/ Spectacular West Facing Views Over Haye...,hayes valley,San Francisco,2022-09-28T15:06:37-0700,2759,1,1,641,None,open house dates friday 2022-10-07 satur...,Reduced Rate on 1x1 at Brand New 100 Van Ness!...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,https://images.craigslist.org/01010_hbtjexHi2I...
7546522181,Beautiful 1BR / Fully Remodeled / WD in Unit /...,russian hill,San Francisco,2022-10-16T22:30:16-0700,4095,1,1,600,2200 Leavenworth,cats are OK - purrr dogs are OK - wooof apa...,"* GET IT WHILE IT'S HOTT, WON'T LAST LONG!! -*...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,https://images.craigslist.org/00l0l_6J3y3Qgxmb...
7547280923,Beautifully Renovated Pac Heights 2BR/1BA; Bri...,pacific heights,San Francisco,2022-10-18T16:28:49-0700,4695,2,1,1002,1801 California,cats are OK - purrr dogs are OK - wooof apa...,"1801 California #202 San Francisco, CA 94109 ...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,https://images.craigslist.org/00t0t_kQd5Dx0w5U...
7547558701,FREE OCTOBER RENT Renovated 3 BR,mission district,san francisco,2022-10-19T11:30:12-0700,3900,3,1,None,shotwell near 20 th,apartment no laundry on site no smoking st...,"571 Shotwell Street San Francisco, CA 94110 - ...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,https://images.craigslist.org/00303_7ADJOwqRcJ...
7538170113,A very nice 1 bedroom with new kitchen and app...,castro / upper market,San Francisco,2022-09-25T09:32:12-0700,2780,1,1,None,Duboce near Pearl,apartment w/d in unit street parking rent ...,"Now showing A large bedroom, living room All ...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,https://images.craigslist.org/00J0J_ablz3MqqGI...


In [5]:
new_listing_df.shape

(227247, 13)

In [6]:
# Concatenate with existing df along rows
dir_folder = '/Users/pandabear/springboard/CapstoneTwoProject/data/interim/'
existing_df = pd.read_csv(dir_folder + 'listing_df_parsed.csv',index_col='listing_id')

combined_df = pd.concat([existing_df, new_listing_df], axis=0)

In [7]:
existing_df.shape

(910735, 13)

In [8]:
combined_df.shape

(1137982, 13)

In [9]:
combined_df.to_csv(r'/Users/pandabear/springboard/CapstoneTwoProject/data/interim/listing_df_parsed.csv', header=True)
print('Saved to csv file')

Saved to csv file
